# `mcts` package

This is an example from the `mcts` (Monte Carlo Tree Search) package repo.

# Imports

In [ ]:
from __future__ import division

from copy import deepcopy
from mcts import mcts
from functools import reduce
import operator

from pandas import DataFrame

# Tic Tac Toe

The game in this example is "naught and crosses" (aka ``tic tac toe", pt. "jogo da velha"). The game is played on a 3 by 3 grid, where players place down one of two marks each turn. One player controls the "x" mark and another controls "o" mark. Each turn, they place down onto the check board either an "x" or an "o", respectively. If any of them is able to place three of their marks consecutively in any direction (vertical, horizontal or diagonal) they win.

## Action class

The `Action` class defines which actions a player can take (i.e. place down an "x" or an "o"). As the game of tic tac toe is played on a 2d grid, an action entails choosing a position on the horizontal (`x`) and vertical (`y`) axes in which to place one's mark (`player`).

In [ ]:
class Action():
    def __init__(self, player, x, y):
        self.player = player
        self.x = x
        self.y = y

    def __str__(self):
        return str((self.x, self.y))

    def __repr__(self):
        return str(self)

    def __eq__(self, other):
        return self.__class__ == other.__class__ and self.x == other.x and self.y == other.y and self.player == other.player

    def __hash__(self):
        return hash((self.x, self.y, self.player))

Class methods:
- `__init__`: instantiates an action object
- `__str__`: converts an action's position (i.e. x and y coordinates) to a character string (`str`) representation
- `__repr__`: converts the action object itself to a character string (`str`) representation
- `__eq__`: equality method for determining if two actions are the same
- `__hash__`: retrieve the object's hash value (i.e. its id as an `int`)

For instance, `Action(1,0,0)` means player 1 places their mark on the grid's (x,y) = (0,0) coordinate.

## Game class

The game class keeps track of the game's current state and defines the rules for choosing a player's next action.

In [ ]:
class NaughtsAndCrossesState():
    def __init__(self):
        self.board = [[0, 0, 0], [0, 0, 0], [0, 0, 0]]
        self.currentPlayer = 1

    def getCurrentPlayer(self):
        return self.currentPlayer

    def getPossibleActions(self):
        possibleActions = []
        for i in range(len(self.board)):
            for j in range(len(self.board[i])):
                if self.board[i][j] == 0:
                    # create an action object for each action
                    possibleActions.append(Action(player=self.currentPlayer, x=i, y=j))
        return possibleActions

    def takeAction(self, action):
        newState = deepcopy(self)
        newState.board[action.x][action.y] = action.player
        newState.currentPlayer = self.currentPlayer * -1
        return newState

    def isTerminal(self):
        for row in self.board:
            if abs(sum(row)) == 3:
                return True
        for column in list(map(list, zip(*self.board))):
            if abs(sum(column)) == 3:
                return True
        for diagonal in [[self.board[i][i] for i in range(len(self.board))],
                         [self.board[i][len(self.board) - i - 1] for i in range(len(self.board))]]:
            if abs(sum(diagonal)) == 3:
                return True
        return reduce(operator.mul, sum(self.board, []), 1)

    def getReward(self):
        for row in self.board:
            if abs(sum(row)) == 3:
                return sum(row) / 3
        for column in list(map(list, zip(*self.board))):
            if abs(sum(column)) == 3:
                return sum(column) / 3
        for diagonal in [[self.board[i][i] for i in range(len(self.board))],
                         [self.board[i][len(self.board) - i - 1] for i in range(len(self.board))]]:
            if abs(sum(diagonal)) == 3:
                return sum(diagonal) / 3
        return False

Class methods:
- `__init__`: instantiates a game state object and defines the board and the current player
- `getCurrentPlayer`: retrieves the current player's id 
- `getPossibleActions`: generates a list of feasible actions (i.e. any non-occupied square)
- `takeAction`: given an action object, places down its player's mark (`action.player`) onto the (`action.x`,`action.y`) coordinates; then updates the current player and returns the new state (new board and new current player)
- `isTerminal`: checks whether a player has own the game (i.e. placed three consecutive marks)
- `getReward`: returns `1` if player `1` has won or `-1` if player `-1` has won

## mcts class

Finally, the `mcts` "searcher" class optimizes a player's next action, given an initial state object (i.e. the "game" class above).

# Example game

Player 1 chooses their action:

In [ ]:
gameState = NaughtsAndCrossesState()
print(f"current player: {gameState.currentPlayer}")
searcher = mcts(timeLimit=1000)
action = searcher.search(initialState=gameState)
print(f"feasible actions: place mark on {gameState.getPossibleActions()} squares")
print(f"player's action: place mark on {action} square")

Player 1 takes their action:

In [ ]:
print(f"current player: {gameState.currentPlayer}")
gameState = gameState.takeAction(action)
print(f"""current board:
{DataFrame(gameState.board)}
"""
)
print(f"current player: {gameState.currentPlayer}")

Player 2 chooses their action:

In [ ]:
print(f"current player: {gameState.currentPlayer}")
searcher = mcts(timeLimit=1000)
action = searcher.search(initialState=gameState)
print(f"feasible actions: place mark on {gameState.getPossibleActions()} squares")
print(f"player's action: place mark on {action} square")

Player 2 takes their action:

In [ ]:
print(f"current player: {gameState.currentPlayer}")
gameState = gameState.takeAction(action)
print(f"""current board:
{DataFrame(gameState.board)}
"""
)
print(f"current player: {gameState.currentPlayer}")

And so on and so forth.

Now, automatically,

In [ ]:
timeMs = 1000
gameState = NaughtsAndCrossesState()

while(len(gameState.getPossibleActions())):
    print(len(gameState.getPossibleActions()))
    print(f"current player: {gameState.currentPlayer}")
    print(f"""current board:
        {DataFrame(gameState.board)}
        """
    )
    gameState = gameState.takeAction(mcts(timeMs).search(gameState))

print("game ended.")

In [29]:
gameState.getPossibleActions()

[(0, 0), (0, 1), (2, 1), (2, 2)]